# Prerequisites

In [1]:
import numpy as np
import cvxpy as cp
import gurobipy as gp
from gurobipy import GRB

## Parameters

In [2]:
# Sets

m = 3 #number of customer locations
n = 8 #number of potential truck locations

C = list(range(1,m+1,1))
L = list(range(1,n+1,1))

np.random.seed(1)

r = 10 # revenue per burrito
k = 5 # cost of ingredients per burrito

d = np.zeros((m,)) # expected demand of each customer location

value = 100*np.random.rand(m)
count = 0
for i in range(m):
    d[i] = value[count]
    count = count +1
    
f = np.zeros((n,)) # fixed cost of placing a truck at location j
count = 0
for j in range(n):
    f[j] = 250
    count = count +1
    
a = np.zeros((m,n))
value = np.random.rand(m*n)
count = 0
for i in range(m):
    for j in range(n):
        a[i,j] = value[count]
        count = count + 1


## Second Stage Constants

B = 500 # Number of scenarios


low = 10*np.random.rand(m)
high = 10*np.random.rand(m)

# d_xi: our random variable xi for demand d
d_xi = np.zeros((m,B))

for i in range(m):
    d_xi[i,:] = np.random.uniform(np.max(d[i]-low[i],0),d[i]+high[i],B)

## Define Model and Variables

In [3]:
M = gp.Model("2SP")
M.ModelSense = GRB.MAXIMIZE
M.setParam('OutputFlag', 0)  # Telling gurobi to not be verbose
M.params.logtoconsole=0

# VARIABLES 
## First Stage Decision variable
x = M.addMVar((n,), vtype = GRB.BINARY, name = "x")

## 2nd stage decision for each potential scenario
y = M.addMVar((m,n,B), vtype = GRB.BINARY, name = "y")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-23


## Objective Function

In [4]:
obj = 0

# sum for Q(y,d_xi)
for i in range(m):
    for j in range(n):
        obj = obj + gp.quicksum( (r-k) * a[i,j] * d_xi[i,b] * y[i,j,b] for b in range(B))
        
# OBJECTIVE 
M.setObjective(obj/B - f.T @ x)

## Define Constraints

In [5]:
# CONSTRAINTS 


M.addConstrs((gp.quicksum(y[i,j,b] for j in range(n)) <= 1 
            for i in range(m) 
            for b in range(B)))

M.addConstrs(y[i,j,b] <= x[j] for i in range(m) for j in range(n) for b in range(B))

{(0, 0, 0): <MConstr () *awaiting model update*>,
 (0, 0, 1): <MConstr () *awaiting model update*>,
 (0, 0, 2): <MConstr () *awaiting model update*>,
 (0, 0, 3): <MConstr () *awaiting model update*>,
 (0, 0, 4): <MConstr () *awaiting model update*>,
 (0, 0, 5): <MConstr () *awaiting model update*>,
 (0, 0, 6): <MConstr () *awaiting model update*>,
 (0, 0, 7): <MConstr () *awaiting model update*>,
 (0, 0, 8): <MConstr () *awaiting model update*>,
 (0, 0, 9): <MConstr () *awaiting model update*>,
 (0, 0, 10): <MConstr () *awaiting model update*>,
 (0, 0, 11): <MConstr () *awaiting model update*>,
 (0, 0, 12): <MConstr () *awaiting model update*>,
 (0, 0, 13): <MConstr () *awaiting model update*>,
 (0, 0, 14): <MConstr () *awaiting model update*>,
 (0, 0, 15): <MConstr () *awaiting model update*>,
 (0, 0, 16): <MConstr () *awaiting model update*>,
 (0, 0, 17): <MConstr () *awaiting model update*>,
 (0, 0, 18): <MConstr () *awaiting model update*>,
 (0, 0, 19): <MConstr () *awaiting model 

## Solve model and print solution

In [6]:
# SOLVING
M.optimize()
result = x.x

print(f"Optimal profit: ${np.round(M.ObjVal,2)}")
print("Optimal solution (truck placements):")

# Solution
for j in range(n):
    print(f"  x[{j}] = {result[j]}")

Optimal profit: $103.73
Optimal solution (truck placements):
  x[0] = -0.0
  x[1] = 0.0
  x[2] = 1.0
  x[3] = 0.0
  x[4] = -0.0
  x[5] = 0.0
  x[6] = -0.0
  x[7] = 0.0


Suppose the third customer location had demand 50 instead of zero.

In [7]:
d[2] = 50

# d_xi: our random variable xi for demand d
d_xi = np.zeros((m,B))

for i in range(m):
    d_xi[i,:] = np.random.uniform(np.max(d[i]-low[i],0),d[i]+high[i],B)

In [8]:
M = gp.Model("2SP")
M.ModelSense = GRB.MAXIMIZE
M.setParam('OutputFlag', 0)  # Telling gurobi to not be verbose
M.params.logtoconsole=0

# VARIABLES 
## First Stage Decision variable
x = M.addMVar((n,), vtype = GRB.BINARY, name = "x")

## 2nd stage decision for each potential scenario
y = M.addMVar((m,n,B), vtype = GRB.BINARY, name = "y")


obj = 0

# sum for Q(y,d_xi)
for i in range(m):
    for j in range(n):
        obj = obj + gp.quicksum( (r-k) * a[i,j] * d_xi[i,b] * y[i,j,b] for b in range(B))
        
# OBJECTIVE 
M.setObjective(obj/B - f.T @ x)

# CONSTRAINTS 


M.addConstrs((gp.quicksum(y[i,j,b] for j in range(n)) <= 1 
            for i in range(m) 
            for b in range(B)))

M.addConstrs(y[i,j,b] <= x[j] for i in range(m) for j in range(n) for b in range(B))

# SOLVING
M.optimize()
result = x.x

print(f"Optimal profit: ${np.round(M.ObjVal,2)}")
print("Optimal solution (truck placements):")

# Solution
for j in range(n):
    print(f"  x[{j}] = {result[j]}")

Optimal profit: $333.88
Optimal solution (truck placements):
  x[0] = -0.0
  x[1] = -0.0
  x[2] = 1.0
  x[3] = 0.0
  x[4] = -0.0
  x[5] = -0.0
  x[6] = -0.0
  x[7] = 0.0
